In [ ]:
import itertools
from functools import reduce
from itertools import product

import operator

K = Fields()

q = 2**61 - 1
Fq = GF(q)
x = polygen(Fq, 'x')
Fq2.<u> = Fq.extension(x^2+1)
log_n = 4
n = 2**log_n
v = vector(Fq, [Fq.random_element() for _ in range(n)])

def get_bit(n, k):
    return (n >> k) & 1

def int_to_bits(n, bit_length=None):
    if bit_length is None:
        bit_length = n.bit_length() if n != 0 else 1
    return [(n >> i) & 1 for i in range(bit_length)]

def multilinear_extension(v, F, var_names=None):
    n = len(v)
    m = n.bit_length() - 1
    if 2 ** m != n:
        raise ValueError("Vector length must be a power of 2.")
    
    if var_names is None:
        var_names = [f'X{i}' for i in range(1, m+1)]
    R = PolynomialRing(F, var_names)
    X = R.gens()
    p = R.zero()
    
    for i in range(n):
        term = F(v[i]) 
        bits = int_to_bits(i,m) 
        
        for j in range(m):
            if bits[j] == 1:
                term *= X[j]
            else:
                term *= (1 - X[j])
        p += term
    return p

v_tilde = multilinear_extension(v, Fq)

for i, t in enumerate(v):
  assert t == v_tilde([j for j in int_to_bits(i, log_n)])


def multilinear_matrix_extension(M, F, var_names=None):
    d, n = M.dimensions()
    total_elements = d * n
    
    m = total_elements.bit_length() - 1
    if 2^m != total_elements:
        raise ValueError("Matrix dimensions must multiply to a power of 2 (d * n = 2^m).")
    
    # Flatten the matrix into a vector (row-major order)
    v = M.list()
    
    if var_names is None:
        var_names = [f'X{i}' for i in range(1, m+1)]
    R = PolynomialRing(F, var_names)
    X = R.gens()
    
    p = R.zero()
    for i in range(total_elements):
        term = F(v[i])  
        bits = int_to_bits(i,m) 
        
        for j in range(m):
            if bits[j] == 1:
                term *= X[j]
            else:
                term *= (1 - X[j])
        
        p += term
    
    return p

M = Matrix(Fq, [[ Fq.random_element() for _ in range(n)] for _ in range(n)])
M_tilde = multilinear_matrix_extension(M, Fq)

for i, row in enumerate(M):
  for j, e in enumerate(row):
    assert M[i][j] == M_tilde([y for y in int_to_bits(j, log_n)]+[x for x in int_to_bits(i, log_n)])


# Build multi-linear basis polynomials
var_names = [f'X{i}' for i in range(log_n)]
R = PolynomialRing(Fq2, var_names)
X = R.gens()
print(X)
# print([X[i] for i in range(log_n)])
basis = []
for i in range(n):
    bits = [(i >> j) & 1 for j in range(log_n)]
    poly = 1
    for b, x in zip(bits, [X[i] for i in range(log_n)]):
        poly *= x if b else (1 - x)
    basis.append(poly)

# print(basis)

tilde_f_r = sum(v[i] * basis[i] for i in range(n))
v_tilde2 = multilinear_extension(v, Fq2)
for i, t in enumerate(v):
  params = [j for j in int_to_bits(i, log_n)]
  assert tilde_f_r(params) == v_tilde(params)


In [2]:
# 2.3 Rings and Modules
eta = 81
d = 54
q = 2**61 - 1
Fq = GF(q)
x = polygen(Fq, 'x')
Phi = x ** 54 + x ** 27 + 1
Rq.<u> = Fq.extension(Phi)
kappa = 16
## m = 2 ** 22 # TOO MUCH MEMORY
#m = 2 ** 8 # for testing purposes
#M = random_matrix(Rq, kappa, m)
a = Rq.random_element()

def cf(a):
    return a.list()

def cf_inv(a):
    return Rq(a)

def shift_matrix(Fq, Phi, d):
    F = matrix(Fq, d, d)
    c = cf(Phi)
    for i in range(d):
        for j in range(d):
            if i - 1 == j:
                F[i, j] = 1
            if j == d - 1:
                F[i, j] = -c[i]
    return F

def rot(a, Phi, Fq):
    cf_a = vector(Fq, a if isinstance(a, list) else cf(a))
    d = len(cf_a)
    F = shift_matrix(Fq, Phi, d)

    columns = [cf_a]
    F_power = F  # Start with F^1
    for _ in range(1, d):
        current = F_power * cf_a
        columns.append(current)
        F_power = F_power * F  # Compute next power incrementally
    rows = list(zip(*columns))
    M = matrix(Fq, len(cf_a), len(cf_a), sum([list(row) for row in rows], []))

    return M    # print([(F**i) * cf_a for i in range(1, d)])
    
# print(shift_matrix(Fq, Phi, d))
rot_a = rot(a, Phi, Fq)
b = Rq.random_element()
assert rot_a * vector(Fq, cf(b)) == vector(Fq,cf(a*b))

# F7 = GF(7)

# y = polygen(F7, 'y')
# Phi_test = y ** 3 + 1
# R7.<u> = F7.extension(Phi_test)

# a = 4*u**2 + 2*u + 3
# rot_a = rot(a, Phi_test, F7)
# print(rot_a)
# b = u**2 + 1
# print("cf(a*b)", cf(a*b))
# print("rot_a * vector(F7, cf(b)", rot_a * vector(F7, cf(b)))
# assert rot_a * vector(F7, cf(b)) == vector(F7, cf(a*b))

# 3.2 Neo’s solution, part-1: A matrix commitment scheme

def embed_Fq_to_Rq(z_i, d):
    """
    Embed a field element z_i ∈ Fq into Rq by using its bits
    as coefficients of a polynomial in Rq.
    """
    z_int = int(z_i)
    bits = [((z_int >> i) & 1) for i in range(d)]
    poly = sum(Fq(bits[i]) * u^i for i in range(len(bits)))
    return Rq(poly)

z_i = Fq.random_element()
z_i_embedded = embed_Fq_to_Rq(z_i, d)

print(f"Original Fq element: {z_i}")
print(f"Embedded Rq element: {z_i_embedded}")


Original Fq element: 712880922976574543
Embedded Rq element: u^53 + u^50 + u^47 + u^45 + u^43 + u^40 + u^38 + u^37 + u^35 + u^34 + u^29 + u^26 + u^24 + u^21 + u^19 + u^17 + u^16 + u^15 + u^14 + u^12 + u^11 + u^10 + u^6 + u^3 + u^2 + u + 1


In [ ]:
from random import randint


def decomp_b(z, d=None):
    F = z.base_ring()
    m = len(z)
    b = 2

    # Determine degree if not provided
    if d is None:
        # Find smallest d such that b^d > max(z)
        max_val = max([abs(int(zi)) for zi in z])
        d = 1
        while b**d <= max_val:
            d += 1

    # Initialize the result matrix
    # Z = matrix(F, d, m)

    # Perform b-ary decomposition
    cols = []
    for j in range(m):
        value = int(z[j])
        bits = [((value >> i) & 1) for i in range(d)]
        cols.append(bits)
    rows = list(zip(*cols))
    Z = matrix(F, d, m, sum([list(row) for row in rows], []))

    return Z


z_1 = Fq(123)
z_2 = Fq(567)
z_3 = Fq(890)
m = 3
z_1_embedded = embed_Fq_to_Rq(z_1, d)
z_2_embedded = embed_Fq_to_Rq(z_2, d)
z_3_embedded = embed_Fq_to_Rq(z_3, d)
z_vec = vector(Fq, [z_1, z_2, z_3])
print(z_vec)
Z = decomp_b(z_vec, d)
print("len(cf(z_1_embedded))", len(cf(z_1_embedded)))
print("len([Z[i, 0] for i in range(d)])", len([Z[i, 0] for i in range(d)]))
assert cf(z_1_embedded) == [Z[i, 0] for i in range(d)]
assert cf(z_2_embedded) == [Z[i, 1] for i in range(d)]
assert cf(z_3_embedded) == [Z[i, 2] for i in range(d)]

print("check sum", [Fq(sum([2**i * Z[i, col] for i in range(d)])) for col in range(m)])
assert list(z_vec) == [
    Fq(sum([2**i * Z[i, col] for i in range(d)])) for col in range(m)
]


def split_b(Z, k=None):
    F = Z.base_ring()
    d, m = Z.dimensions()
    b = 2

    # Determine depth if not provided
    if k is None:
        # Find smallest k such that b^k > max(Z)
        max_val = max([abs(int(Z[i, j])) for i in range(d) for j in range(m)])
        k = 1
        while b**k <= max_val:
            k += 1
    # Initialize the result matrices
    result = [matrix(F, d, m) for _ in range(k)]

    # Perform b-ary decomposition
    for i in range(d):
        for j in range(m):
            value = int(Z[i, j])
            bits = [((value >> i) & 1) for i in range(k)]
            for l in range(k):
                result[l][i, j] = F(bits[l])
    return result


max_val = 2**20  # low norm
test = matrix(Fq, d, m)
for i in range(d):
    for j in range(m):
        test[i, j] = Fq(randint(0, max_val - 1))
Z = split_b(test)
Z_restored = matrix(Fq, d, m)
for i, z in enumerate(Z):
    Z_restored += (2**i) * z

print(Z_restored)
assert test == Z_restored

(123, 567, 890)
len(cf(z_1_embedded)) 54
len([Z[i, 0] for i in range(d)]) 54
check sum [123, 567, 890]
[1011024  201008  236249]
[ 204330  397535  336977]
[ 705318  234151  661837]
[  67695  832648  903038]
[ 568287  122775  646073]
[ 625866  330636  809317]
[ 869065  408701  447136]
[ 980796  386635  833948]
[ 162655  615627  615279]
[ 427685  324569  625908]
[ 256586  489316  130744]
[ 185095  149747  908251]
[ 579943  746845  582458]
[ 652115 1041501  784897]
[ 989809   43660   66458]
[ 946601  194937  659040]
[ 453262  788507  844324]
[ 427356  872969 1046167]
[ 658668  653847  328414]
[  64503   59700  742153]
[1040502   50820  373813]
[ 812768  273076  872821]
[ 113424  920154  261419]
[ 202185  125309  478781]
[    395  877636   11380]
[ 616109  487178  528210]
[ 731684  375318  778380]
[ 649468  406032  738563]
[ 201346       4 1032729]
[ 259449  459249  531669]
[ 443426  630706  600352]
[ 463083  124047  244154]
[1032244   17801  632389]
[  22130  190373  589171]
[ 268006  760

In [ ]:
print("check sum", [sum(b**row * [Z[row, i] for i in range(d)]) for row in range(3)])